## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [13]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.SimpleRNN(32)
        # self.rnn = tf.keras.layers.LSTM(32)
        # self.rnn = tf.keras.layers.GRU(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 학습, 테스트 루프 정의

In [14]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [15]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=32)

x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [16]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [17]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.6037062406539917, Accuracy: 64.3239974975586, Test Loss: 0.4819228947162628, Test Accuracy: 76.71199798583984
Epoch 2, Loss: 0.38026708364486694, Accuracy: 83.06000518798828, Test Loss: 0.4789382517337799, Test Accuracy: 77.37200164794922
Epoch 3, Loss: 0.22083526849746704, Accuracy: 91.32400512695312, Test Loss: 0.5991339087486267, Test Accuracy: 76.66799926757812
Epoch 4, Loss: 0.09156794100999832, Accuracy: 96.9000015258789, Test Loss: 0.8243484497070312, Test Accuracy: 74.68399810791016
Epoch 5, Loss: 0.037218425422906876, Accuracy: 98.87200164794922, Test Loss: 1.054952621459961, Test Accuracy: 72.39999389648438
Epoch 6, Loss: 0.019787926226854324, Accuracy: 99.36399841308594, Test Loss: 1.231892704963684, Test Accuracy: 72.43199920654297
Epoch 7, Loss: 0.01577329821884632, Accuracy: 99.5199966430664, Test Loss: 1.331209421157837, Test Accuracy: 74.6760025024414
Epoch 8, Loss: 0.0169827863574028, Accuracy: 99.43600463867188, Test Loss: 1.373236060142517, Test Accu